In [1]:
# Load Libraries
import json
import pandas as pd
import numpy as np
from pathlib import Path

# Load project configuration
with open('config.json', 'r') as f:
    config = json.load(f)

# Extract paths
data_processed = Path(config['paths']['data_processed'])

In [2]:
# Define WBGT Thresholds
wbgt_thresholds = {
'caution': 26.0,
'high_risk': 28.0,
'severe_risk': 30.0
}

# Define risk categories
risk_categories = {
    'normal': {'min': 0, 'max': 26, 'color': '#2ECC71', 'label': 'Normal'},
    'caution': {'min': 26, 'max': 28, 'color': '#F39C12', 'label': 'Caution'},
    'high_risk': {'min': 28, 'max': 30, 'color': '#E74C3C', 'label': 'High Risk'},
    'severe_risk': {'min': 30, 'max': 50, 'color': '#8B0000', 'label': 'Severe Risk'}
}

# Analysis period (most recent 5 years available)
analysis_period = {
    'start_year': 2016,
    'end_year': 2020,
}

# Historical baseline for trend analysis
baseline_period = {
    'start_year': 1982,
    'end_year': 2020,
}

# Summer months (when heat risk is highest)
summer_months = [6, 7, 8]  # June, July, August

In [3]:
# Social vulnerability indicators (Simplified Housing Criteria with CDC SVI weights not considering transportation due to scope of project)

social_vulnerability_indicators = {
    # Socioeconomic Status
    'poverty_rate': {'weight': 0.0825, 'direction': 'higher_worse'},
    'unemployment_rate': {'weight': 0.0825, 'direction': 'higher_worse'},
    'per_capita_income': {'weight': 0.0825, 'direction': 'lower_worse'},  # invert: lower income = worse
    'no_high_school_pct': {'weight': 0.0825, 'direction': 'higher_worse'},

    # Household Composition
    'elderly_pct': {'weight': 0.0833, 'direction': 'higher_worse'},
    'children_pct': {'weight': 0.0833, 'direction': 'higher_worse'},
    'single_parent_pct': {'weight': 0.0833, 'direction': 'higher_worse'},
    'disability_pct': {'weight': 0.0833, 'direction': 'higher_worse'},  # replaced vehicles

    # Minority Status
    'minority_pct': {'weight': 0.10, 'direction': 'higher_worse'},
    'limited_english_pct': {'weight': 0.10, 'direction': 'higher_worse'},

    # Housing
    'multi_unit_pct': {'weight': 0.0733, 'direction': 'higher_worse'},
    'crowding_pct': {'weight': 0.0733, 'direction': 'higher_worse'},
}

In [4]:
# Infrastructure accessibility indicators

infrastructure_thresholds = {
    'cooling_centers': {
        'ideal_distance_ft': 1320,      # 1/4 mile 
        'max_distance_ft': 2640,        # 1/2 mile
        'weight': 0.25,
    },
    'green_spaces': {
        'ideal_distance_ft': 1320,      # 1/4 mile
        'max_distance_ft': 2640,        # 1/2 mile
        'weight': 0.20,
    },
    'tree_canopy': {
        'low_coverage_pct': 10,          # < 10% is low
        'high_coverage_pct': 30,         # > 30% is good
        'weight': 0.25,
    },
    'water_features': {
        'ideal_distance_ft': 2640,      # 1/2 mile
        'max_distance_ft': 5280,        # 1 mile
        'weight': 0.15,
    }, 
    'urban_design': {
        'high_avg_height_m': 30,      # >30 m (~10 stories) = heat trapping
        'low_avg_height_m': 10,       # <10 m (~3 stories) = better ventilation
        'high_impervious_pct': 80,    # >80% impervious surface = poor
        'low_impervious_pct': 50,     # <50% impervious surface = good
        'weight': 0.15,
    }
    }

In [5]:
# Configure Composite Heat Vulnerability Index (CHVI)
chvi_weights = {
    'heat_exposure': 0.35,
    'social_vulnerability': 0.35,
    'infrastructure_deficit': 0.30  
}

In [6]:
# Normalization settings
normalization_config = {
    'method': 'min-max',
    'clip_outliers': True,
    'outlier_threshold': 3.0, # Number of standard deviations
    'description': 'Min-max normalization scales all indicators to 0-1 range'
}

In [7]:
# Define Historical Redlining Categories with 1930s HOLC (Home Owners' Loan Corporation) grade mapping
holc_grades = {
    'A': {'name': 'Best', 'color': '#2ECC71', 'risk_multiplier': 0.5},
    'B': {'name': 'Still Desirable', 'color': '#3498DB', 'risk_multiplier': 0.75},
    'C': {'name': 'Declining', 'color': '#F39C12', 'risk_multiplier': 1.0},
    'D': {'name': 'Hazardous', 'color': '#E74C3C', 'risk_multiplier': 1.5},
    'None': {'name': 'Not Graded', 'color': '#95A5A6', 'risk_multiplier': 1.0}
}

In [8]:
# Save Configurations
analysis_config = {
    'wbgt_thresholds': wbgt_thresholds,
    'risk_categories': risk_categories,
    'time_periods': {
        'analysis': analysis_period,
        'baseline': baseline_period,
        'summer_months': summer_months
    },
    'social_vulnerability': social_vulnerability_indicators,
    'infrastructure': infrastructure_thresholds,
    'chvi_weights': chvi_weights,
    'normalization': normalization_config,
    'holc_grades': holc_grades
}

config_file = data_processed / 'analysis_config.json'
with open(config_file, 'w') as f:
    json.dump(analysis_config, f, indent=4)